In [1]:
from datacentric import config
from google.cloud import aiplatform

cfg = config.new()
aiplatform.init(project=cfg.project_id, location=cfg.location)

In [39]:
DATASET_URI = "gs://ruinard_datacentric/assets/data/*.jsonl"
display_name = 'vertex_metadata_managed_question_answering_dataset'

dataset_artifact = aiplatform.Artifact.create(
    schema_title="system.Dataset", display_name=display_name, uri=DATASET_URI, metadata={"custom_schema": "gs://my_bucket/my_schema/schema.yaml"}
)

In [40]:
dataset_artifact.to_dict()

{'name': 'projects/714561025480/locations/us-central1/metadataStores/default/artifacts/54ff5787-8922-4921-887a-615dcceeaa6e',
 'displayName': 'vertex_metadata_managed_question_answering_dataset',
 'uri': 'gs://ruinard_datacentric/assets/data/*.jsonl',
 'etag': '1734526043195',
 'createTime': '2024-12-18T12:47:23.195Z',
 'updateTime': '2024-12-18T12:47:23.195Z',
 'state': 'LIVE',
 'schemaTitle': 'system.Dataset',
 'schemaVersion': '0.0.1',
 'metadata': {'custom_schema': 'gs://my_bucket/my_schema/schema.yaml'}}

In [41]:
ds = aiplatform.Artifact.list(filter=f"display_name={display_name}")[0]

In [42]:
bucket_name, blob_path = ds.uri.split('/', 3)[2:]
bucket_name, blob_path

('ruinard_datacentric', 'assets/data/*.jsonl')

[<Blob: ruinard_datacentric, assets/data/question_answering.jsonl, 1734525992023042>,
 <Blob: ruinard_datacentric, assets/data/questions_only.jsonl, 1734525992352562>]

In [44]:
import json
from google.cloud import storage

client = storage.Client()
bucket = client.get_bucket(bucket_name)
# get all blobs which match wildcard patterh
blobs = list(bucket.list_blobs(match_glob=blob_path))
data = []
for blob in blobs:
    jsonl_data = blob.download_as_string().decode('utf-8').split('\n')
    for line in jsonl_data:
        data.append(json.loads(line))
    


In [45]:
data

[{'question': 'What is the capital of France?', 'answer': 'Paris'},
 {'question': 'What is the largest planet in our solar system?',
  'answer': 'Jupiter'},
 {'question': "Who wrote 'To Kill a Mockingbird'?", 'answer': 'Harper Lee'},
 {'question': 'What is the boiling point of water?',
  'answer': '100 degrees Celsius'},
 {'question': 'What is the currency of Japan?', 'answer': 'Yen'},
 {'question': 'What is the capital of France?'},
 {'question': 'What is the largest planet in our solar system?'},
 {'question': "Who wrote 'To Kill a Mockingbird'?"},
 {'question': 'What is the boiling point of water?'},
 {'question': 'What is the currency of Japan?'}]

In [ ]:
aiplatform.Artifact.list()